In [39]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
from easymore import Easymore


# easymore, creating the remap for mizuRoute merit no lake

In [40]:

# create the case names and shapefile for each case and loop over them to create the reampping file

case_names = ['South_Sasktoon_merit_no_lake',
              'South_Sasktoon_merit_lake',
              'South_Sasktoon_HDMA_no_lake',
              'South_Sasktoon_HDMA_lake']
shapefile_names = ['../data/shp/cat_71021602_merit.shp',
                   '../data/shp/cat_lake_area_corrected_71021602_merit.shp',
                   '../data/shp/cat_3044094_HDMA.shp',
                   '../data/shp/cat_lake_area_corrected_3044094_merit.shp']
shp_id_names =    ['cat_id','cat_id', 'hruid', 'ID']


for i in np.arange(len(case_names)):

    # initializing EASYMORE object
    esmr = Easymore()

    # specifying EASYMORE objects
    # name of the case; the temporary, remapping and remapped file names include case name
    esmr.case_name                = case_names[i]             
    # temporary path that the EASYMORE generated GIS files and remapped file will be saved
    esmr.temp_dir                 = './temporary/'
    # name of target shapefile that the source netcdf files should be remapped to
    esmr.target_shp               = shapefile_names[i] # cats and lakes
    esmr.target_shp_ID            = shp_id_names[i] # 
    # 
    esmr.source_nc                = '../mizuRoute/input/mizuRoute_mswep_v1_1980_subset.nc'
    # name of variables from source netCDF file(s) to be remapped
    esmr.var_names                = ['prec']
    # name of variable longitude in source netCDF files
    esmr.var_lon                  = 'lon'
    # name of variable latitude in source netCDF files
    esmr.var_lat                  = 'lat'
    # name of variable time in source netCDF file; should be always time
    esmr.var_time                 = 'time'
    # location where the remapped netCDF file will be saved
    esmr.output_dir               = './output/'
    # format of the variables to be saved in remapped files,
    # if one format provided it will be expanded to other variables
    esmr.format_list              = ['f4']
    # fill values of the variables to be saved in remapped files,
    # if one value provided it will be expanded to other variables
    esmr.fill_value_list          = ['-99999.00']
    # if required that the remapped values to be saved as csv as well
    esmr.only_create_remap_csv    = True

    # execute EASYMORE
    esmr.nc_remapper()

    # saving the remapp as nc file
    file_to_save = '../data/rempping_file/'+esmr.case_name+'_remapping.nc'

    # merge the remapping and save as a netcdf file
    df = pd.read_csv('./temporary/'+esmr.case_name+'_remapping.csv')

    # sort and get the case
    df = df.sort_values(by=['ID_t']) # sort based on ID_t
    case = df['easymore_case'].iloc[0].item()

    # river network ID and its frequency in intersection with runoff field grid
    RN_id, RN_frequency_in_intersection = np.unique(df['ID_t'], return_counts=True)
    print("the dimension of overlap between river network and hydrological units:")
    print(len(df))
    if len(RN_id) == len(RN_frequency_in_intersection):
        print("the dimension of river network and its frequancy are:")
        print(len(RN_id))
        print(len(RN_frequency_in_intersection))
    else:
        sys.exit("the dimension of river network and its frequency are not the same")

    # get remapping information from easymore remap file
    IDmask  = np.array(df['ID_t']) # the ID of river network
    weight  = np.array(df['weight']) # the weight of each hydrological unit in river network
    i_index = np.array(df['cols']) # cols for case 1 and 2
    j_index = np.array(df['rows']) # rows for case 1 and 2
    ID_s    = np.array(df['ID_s']) # ID from unstructure mesh case 3


    # write the netcdf file
    if os.path.isfile(file_to_save):
        os.remove(file_to_save)

    with nc4.Dataset(file_to_save, "w", format="NETCDF4") as ncid: # creating the NetCDF file

        # define the dimensions
        dimid_ID  = ncid.createDimension('polyid', len(RN_id))  # limited dimensiton equal the number of hruID
        dimid_RMP = ncid.createDimension('intersect', len(df))   # intersection

        # Variables RN_id, RN_frequency_in_intersection
        #
        RN_IDvar   = ncid.createVariable('polyid', 'int',  ('polyid', ), fill_value = -9999, zlib=True, complevel=9)
        RN_IDvar.long_name = 'ID of River Network subbasins'
        RN_IDvar.standard_name = 'ID of River Network subbasins'
        RN_IDvar.units = '1'
        RN_IDvar[:] = RN_id
        #
        RN_FRvar   = ncid.createVariable('frequency', 'int', ('polyid', ), fill_value = -9999, zlib=True, complevel=9)
        RN_FRvar.long_name = 'Frequancy of intersection River Network subbasins with hydrological subbasins'
        RN_FRvar.standard_name = 'Frequancy of intersection River Network subbasins with hydrological subbasins'
        RN_FRvar.units = '1'
        RN_FRvar[:] = RN_frequency_in_intersection

        # ID_mask, weight, i_index, j_index, ID_s
        #
        IDmaskvar  = ncid.createVariable('IDmask', 'int',  ('intersect', ), fill_value = -9999, zlib=True, complevel=9)
        IDmaskvar.long_name = 'ID of rive network subbasins'
        IDmaskvar.standard_name = 'ID of rive network subbasins'
        IDmaskvar.units = '1'
        IDmaskvar[:] = IDmask
        #
        weightvar   = ncid.createVariable('weight', 'f8',  ('intersect', ), fill_value = -9999, zlib=True, complevel=9)
        weightvar.long_name = 'weight of each hydrological unit in river network subbasins'
        weightvar.standard_name = 'weight of each hydrological unit in river network subbasins'
        weightvar.units = '1'
        weightvar[:] = weight

        if case == 1 or case == 2:
            #
            i_indexvar  = ncid.createVariable('i_index', 'int',  ('intersect', ), fill_value = -9999, zlib=True, complevel=9)
            i_indexvar.long_name = 'cols from the source nc file'
            i_indexvar.standard_name = 'cols from the source nc file'
            i_indexvar.units = '1'
            i_indexvar[:] = i_index + 1
            #
            j_indexvar   = ncid.createVariable('j_index', 'int',  ('intersect', ), fill_value = -9999, zlib=True, complevel=9)
            j_indexvar.long_name = 'rows from the source nc file'
            j_indexvar.standard_name = 'rows from the source nc file'
            j_indexvar.units = '1'
            j_indexvar[:] = j_index + 1

        if case == 3:
            #
            ID_HRvar   = ncid.createVariable('ID_HR', 'int',  ('intersect', ), fill_value = -9999, zlib=True, complevel=9)
            ID_HRvar.long_name = 'river network ID'
            ID_HRvar.standard_name = 'river network ID'
            ID_HRvar.units = '1'
            ID_HRvar[:] = ID_s

        # general attributes for NetCDF file
        ncid.Conventions = 'CF-1.6'
        ncid.Author = 'The data were written by easymore_codes'
        ncid.License = 'MIT'
        ncid.History = 'Created '
        ncid.Source = 'Case: ; remapped by script from library of Shervan Gharari (https://github.com/ShervanGharari/EASYMORE).'


    # show the remap file
    ds = xr.open_dataset(file_to_save)
    ds

EASYMORE version 1.1.2 is initiated.
no author name is provided. The author name is changed to (author name)!
remap file is provided; EASYMORE will use this file and skip creation of remapping file
EASYMORE will remap variable  prec  from source file to variable  prec  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dimensions of the variables and latitude and longitude
EASYMORE detects that all the variables have dimensions of:
['time', 'lat', 'lon']
EASYMORE detects that the longitude variables has dimensions of:
['lon']
EASYMORE detects that the latitude variables has dimensions of:
['lat']
--CREATING-REMAPPING-FILE--
Started at date and time 2023-08-30 11:36:26.211451
EASYMORE detects that target shapefile is in WGS84 (epsg:4326)
EASYMORE detects that the field for ID is provided in sink/target shapefile
EASYMORE detects that either of the fields for latitude or longitude is not provided in sink/target shapefile
calculating centroid

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/easymore/remapper.py:421: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files


Ended at date and time 2023-08-30 11:36:57.313152
It took 31.101701 seconds to finish creating of the remapping file
---------------------------
The flag to create only remap file is True
The remapping file is either created or given to EASYMORE
The remapping Located here:  ./temporary/South_Sasktoon_merit_no_lake_remapping.csv
the dimension of overlap between river network and hydrological units:
5856
the dimension of river network and its frequancy are:
2993
2993
EASYMORE version 1.1.2 is initiated.
no author name is provided. The author name is changed to (author name)!
remap file is provided; EASYMORE will use this file and skip creation of remapping file
EASYMORE will remap variable  prec  from source file to variable  prec  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dimensions of the variables and latitude and longitude
EASYMORE detects that all the variables have dimensions of:
['time', 'lat', 'lon']
EASYMORE detects that th

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/easymore/remapper.py:421: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files


Ended at date and time 2023-08-30 11:37:29.353965
It took 31.939303 seconds to finish creating of the remapping file
---------------------------
The flag to create only remap file is True
The remapping file is either created or given to EASYMORE
The remapping Located here:  ./temporary/South_Sasktoon_merit_lake_remapping.csv
the dimension of overlap between river network and hydrological units:
5962
the dimension of river network and its frequancy are:
3062
3062
EASYMORE version 1.1.2 is initiated.
no author name is provided. The author name is changed to (author name)!
remap file is provided; EASYMORE will use this file and skip creation of remapping file
EASYMORE will remap variable  prec  from source file to variable  prec  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dimensions of the variables and latitude and longitude
EASYMORE detects that all the variables have dimensions of:
['time', 'lat', 'lon']
EASYMORE detects that the l

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/easymore/remapper.py:421: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files


Ended at date and time 2023-08-30 11:37:41.008383
It took 11.552942 seconds to finish creating of the remapping file
---------------------------
The flag to create only remap file is True
The remapping file is either created or given to EASYMORE
The remapping Located here:  ./temporary/South_Sasktoon_HDMA_no_lake_remapping.csv
the dimension of overlap between river network and hydrological units:
1332
the dimension of river network and its frequancy are:
283
283
EASYMORE version 1.1.2 is initiated.
no author name is provided. The author name is changed to (author name)!
remap file is provided; EASYMORE will use this file and skip creation of remapping file
EASYMORE will remap variable  prec  from source file to variable  prec  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dimensions of the variables and latitude and longitude
EASYMORE detects that all the variables have dimensions of:
['time', 'lat', 'lon']
EASYMORE detects that the l

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/easymore/remapper.py:421: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files


Ended at date and time 2023-08-30 11:37:53.485492
It took 12.419223 seconds to finish creating of the remapping file
---------------------------
The flag to create only remap file is True
The remapping file is either created or given to EASYMORE
The remapping Located here:  ./temporary/South_Sasktoon_HDMA_lake_remapping.csv
the dimension of overlap between river network and hydrological units:
1357
the dimension of river network and its frequancy are:
290
290
